# ***CLASSIFICATION ON CIFAR 10 USING PRE_TRAINED MODEL***

In [37]:
#Importing LIbraries
from keras.datasets import cifar10
import numpy as np
import matplotlib.pyplot as plt
import tensorflow
import keras
from keras.layers import Dense
from keras.layers import MaxPool2D,MaxPooling2D
from keras.layers import Conv2D
from keras.layers import Flatten
from keras.utils import np_utils
from keras.layers import Dropout
from keras.optimizers import *
from keras.models import Sequential
from keras.layers import Activation
from keras.optimizers import gradient_descent_v2
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
import cv2


In [23]:
batch_size=32
epoch=1
num_classes=10
learning_rate=1e-4
momentum=0.9

In [39]:
#we import scipy to resize our images quickly
import scipy.misc
import numpy as np

(x_train,y_train),(x_test,y_test)=cifar10.load_data()
x_train=np.array([cv2.resize(x,(48,48),interpolation=cv2.INTER_AREA) for x in x_train])
x_test=np.array([cv2.resize(x,(48,48),interpolation=cv2.INTER_AREA) for x in x_test])

In [40]:
#OneHot For Y_train and Y_test
num_classes=10
y_train = tensorflow.keras.utils.to_categorical(y_train, num_classes)
y_test =  tensorflow.keras.utils.to_categorical(y_test, num_classes)

print(y_train.shape)
print(y_test.shape)

(50000, 10)
(10000, 10)


# **import our pre_trained Model But without the 'Head' Layer**

In [41]:
base_model=vgg.VGG16(weights='imagenet',include_top=False,input_shape=(48,48,3))

In [42]:
#extract the last layer from third block of vgg16 model
last=base_model.get_layer('block3_pool').output

In [43]:
#Add Classification layer on top of it
#x is our model that we are puting on top part of our pre_trained  model 
x=GlobalAveragePooling2D()(last)
x=BatchNormalization()(x)
x=Dense(256,activation='relu')(x)
x=Dense(256,activation='relu')(x)
x=Dropout(0.5)(x)

#we make our top model
number_of_classes=10
top=Dense(num_classes,activation='softmax')(x)



In [44]:
 #construct our full model
 model =keras.Model(inputs=base_model.inputs,outputs=top)


In [45]:
  #now we just iterrate through our base model to  'freez' the layers so that we dont train them 
  for layer in base_model.layers:
        layer.trainable = False

In [46]:
#lets compile our merged model
model.compile(optimizer='Adam', loss='binary_crossentropy',
                 metrics=['accuracy'])

In [47]:
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 48, 48, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 48, 48, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 48, 48, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 24, 24, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 24, 24, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 24, 24, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 12, 12, 128)       0   

In [49]:
#prepare Data augumentation
train_datagen = ImageDataGenerator(
        rescale=1./255,
        horizontal_flip=False)
train_datagen.fit(x_train)
train_generator = train_datagen.flow(x_train,y_train,batch_size=32)



validation_datagen = ImageDataGenerator(
        rescale=1./255,
        horizontal_flip=False)
validation_datagen.fit(x_test)

validation_generator = validation_datagen.flow(x_test,y_test,batch_size=32)



# **Training our Model**

In [50]:
train_step_per_epoch=x_train.shape[0] // batch_size
validation_per_epoch=x_test.shape[0] // batch_size

history=model.fit(train_generator,steps_per_epoch=train_step_per_epoch,validation_data=validation_generator,validation_steps=validation_per_epoch,
                  epochs=3,verbose=1)
                  
                  

Epoch 1/3
1562/1562 [==============================] - 701s 449ms/step - loss: 0.1877 - accuracy: 0.6035 - val_loss: 0.1424 - val_accuracy: 0.6955
Epoch 2/3
1562/1562 [==============================] - 688s 440ms/step - loss: 0.1512 - accuracy: 0.6851 - val_loss: 0.1337 - val_accuracy: 0.7132
Epoch 3/3
1562/1562 [==============================] - 689s 441ms/step - loss: 0.1397 - accuracy: 0.7095 - val_loss: 0.1269 - val_accuracy: 0.7342


In [35]:
model.save("Cifar10_vgg16.h5") 